# Variables categóricas

Hay muchos datos no numéricos por ahí. Aquí se explica cómo usarlo para el aprendizaje automático.

## Introducción
Una **variable categórica** toma sólo un número limitado de valores.

* Considere una encuesta que le pregunte con qué frecuencia desayuna y ofrezca cuatro opciones: "Nunca", "Rara vez", "La mayoría de los días" o "Todos los días". En este caso, los datos son categóricos, porque las respuestas se clasifican en un conjunto fijo de categorías.
* Si las personas respondieran a una encuesta sobre qué marca de automóvil poseen, las respuestas se clasificarían en categorías como "Honda", "Toyota" y "Ford". En este caso, los datos también son categóricos.

Recibirá un error si intenta conectar estas variables en la mayoría de los modelos de aprendizaje automático en Python sin preprocesarlas primero. En este tutorial, compararemos tres enfoques que puede utilizar para preparar sus datos categóricos.

## Tres enfoques

### 1) Eliminar variables categóricas
El enfoque más sencillo para tratar con variables categóricas es simplemente eliminarlas del conjunto de datos. Este enfoque sólo funcionará bien si las columnas no contienen información útil.

## 2) Codificación ordinal
La codificación ordinal asigna cada valor único a un número entero diferente.

Este enfoque supone un orden de las categorías: "Nunca" (0) < "Rara vez" (1) < "La mayoría de los días" (2) < "Todos los días" (3).

Esta suposición tiene sentido en este ejemplo, porque existe una clasificación indiscutible de las categorías. No todas las variables categóricas tienen un orden claro en los valores, pero nos referimos a aquellas que lo tienen como variables ordinales. Para los modelos basados en árboles (como árboles de decisión y bosques aleatorios), se puede esperar que la codificación ordinal funcione bien con variables ordinales.

### 3) Codificación One-Hot
La codificación one-hot crea nuevas columnas que indican la presencia (o ausencia) de cada valor posible en los datos originales. Para entender esto, trabajaremos con un ejemplo.

En el conjunto de datos original, "Color" es una variable categórica con tres categorías: "Rojo", "Amarillo" y "Verde". La codificación one-hot correspondiente contiene una columna para cada valor posible y una fila para cada fila del conjunto de datos original. Siempre que el valor original fuera "Rojo", ponemos un 1 en la columna "Rojo"; si el valor original era "Amarillo", ponemos un 1 en la columna "Amarillo", y así sucesivamente.

A diferencia de la codificación ordinal, la codificación one-hot no supone un ordenamiento de las categorías. Por lo tanto, puede esperar que este enfoque funcione particularmente bien si no hay un orden claro en los datos categóricos (por ejemplo, "Rojo" no es ni más ni menos que "Amarillo"). Nos referimos a las variables categóricas sin una clasificación intrínseca como variables nominales.

La codificación one-hot generalmente no funciona bien si la variable categórica toma una gran cantidad de valores (es decir, generalmente no la usará para variables que toman más de 15 valores diferentes).

### Ejemplo
Como en el tutorial anterior, trabajaremos con el conjunto de datos de Melbourne Housing.

No nos centraremos en el paso de carga de datos. En cambio, puedes imaginar que estás en un punto en el que ya tienes los datos de entrenamiento y validación en X_train, X_valid, y_train e y_valid.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('/kaggle/input/melbourne-housing-snapshot/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Echamos un vistazo a los datos de entrenamiento con el método head() a continuación.

In [2]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


A continuación, obtenemos una lista de todas las variables categóricas en los datos de entrenamiento.

Hacemos esto verificando el tipo de datos (o tipo d) de cada columna. El tipo de objeto indica que una columna tiene texto (en teoría, hay otras cosas que podrían ser, pero eso no es importante para nuestros propósitos). Para este conjunto de datos, las columnas con texto indican variables categóricas.

In [3]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


### Definir función para medir la calidad de cada enfoque

Definimos una función score_dataset() para comparar los tres enfoques diferentes para tratar con variables categóricas. Esta función informa el error absoluto medio (MAE) de un modelo de bosque aleatorio. En general, queremos que el MAE sea lo más bajo posible.

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Puntuación del método 1 (eliminar variables categóricas)
Quitamos las columnas de objetos con el método select_dtypes().

In [6]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
175703.48185157913


### Puntuación del método 2 (codificación ordinal)

Scikit-learn tiene una clase OrdinalEncoder que se puede utilizar para obtener codificaciones ordinales. Recorremos las variables categóricas y aplicamos el codificador ordinal por separado a cada columna.

In [7]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
165936.40548390493


En la celda de código anterior, para cada columna, asignamos aleatoriamente cada valor único a un número entero diferente. Este es un enfoque común que es más sencillo que proporcionar etiquetas personalizadas; sin embargo, podemos esperar un aumento adicional en el rendimiento si proporcionamos etiquetas mejor informadas para todas las variables ordinales.

### Puntuación del método 3 (codificación One-Hot)

Usamos la clase OneHotEncoder de scikit-learn para obtener codificaciones one-hot. Hay una serie de parámetros que se pueden utilizar para personalizar su comportamiento.

* Configuramos handle_unknown='ignore' para evitar errores cuando los datos de validación contienen clases que no están representadas en los datos de entrenamiento, y
* establecer sparse=False garantiza que las columnas codificadas se devuelvan como una matriz numerosa (en lugar de una matriz dispersa).

Para usar el codificador, proporcionamos solo las columnas categóricas que queremos que se codifiquen. Por ejemplo, para codificar los datos de entrenamiento, proporcionamos X_train[object_cols]. (object_cols en la celda de código siguiente es una lista de los nombres de columnas con datos categóricos, por lo que X_train[object_cols] contiene todos los datos categóricos en el conjunto de entrenamiento).

In [9]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


166089.4893009678


## ¿Qué enfoque es mejor?

En este caso, eliminar las columnas categóricas (Método 1) obtuvo el peor resultado, ya que obtuvo la puntuación MAE más alta. En cuanto a los otros dos enfoques, dado que los puntajes MAE devueltos tienen un valor tan cercano, no parece haber ningún beneficio significativo para uno sobre el otro.

En general, la codificación one-hot (método 3) suele tener el mejor rendimiento, y eliminar las columnas categóricas (método 1) suele tener el peor rendimiento, pero varía según el caso.

### Conclusión

El mundo está lleno de datos categóricos. ¡Serás un científico de datos mucho más eficaz si sabes cómo utilizar este tipo de datos común!